In [ ]:
from google.colab import drive 
drive.mount('/content/drive/') 

Mounted at /content/drive/


In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

--2022-12-21 19:57:36--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M  93.5MB/s    in 0.9s    

2022-12-21 19:57:37 (93.5 MB/s) - ‘mini.sh’ saved [89817099/89817099]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8

In [ ]:
# Reload the web page and execute this cell
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.8.15 (default, Nov 24 2022, 15:19:38) 
[GCC 11.2.0]


In [ ]:
!pip install torch==1.10.0 torchvision==0.10.0 --extra-index-url https://download.pytorch.org/whl/cu112
!pip install scikit_learn==1.1.2
!pip install tensorly
!pip install tensorly_torch==0.3.0
!pip install typing_extensions==3.10.0.2
!pip install tensorboard

In [ ]:
# add path
import os
os.chdir("/content/drive/MyDrive/ortho_TONN")
import sys; 
sys.path.append('/content/drive/MyDrive/ortho_TONN/tensor_layers')
sys.path.insert(0, '..')

os.chdir ("./tensor_layers")
!pip install -e .
os.chdir ("../")
tensor_type = 'TensorTrainMatrix'
for kl_mult in [5e-5,1e-5,5e-6]:
  !python train.py   --model-type ${tensor_type} --rank 20  --kl-multiplier $kl_mult --rank-loss True | tee logs/${tensor_type}_${kl_mult}.txt

# **Load Dataset**

In [ ]:
# add path
import os
os.chdir("/content/drive/MyDrive/ortho_TONN")
import sys; 
sys.path.append('/content/drive/MyDrive/ortho_TONN/tensor_layers')
sys.path.insert(0, '..')

# Load Dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets
from torchvision.transforms import ToTensor

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.optim.lr_scheduler import StepLR
from utils import train,test,get_net,OrthoTONN
import time

# Enable this and writer ovject to track metrics during training
from torch.utils.tensorboard import SummaryWriter 

DTYPE = torch.FloatTensor
LONG = torch.LongTensor

# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

/usr/local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Training Params

In [ ]:
# Training parameters
epochs = 10
model_path_dir = './tt_models/'

patience = 10

factor_lr = 0.003
lr = 0.001

r = 8
batch_sz = 64
decay = 0.001
cuda = 1

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32

complete = True
curr_patience = patience

## Model Structure Test

In [ ]:
#def __init__(self,tensor_type,max_rank,dropouts=0.5,prior_type='log_uniform',device=None,dtype=None):
model = OrthoTONN(tensor_type='TensorTrainMatrix',max_rank=2,dropouts=0.5,prior_type='log_uniform',eta=1.0,device=device,dtype=dtype)
model

In [ ]:
# Open tensorboard
%load_ext tensorboard
%tensorboard --logdir=tt_runs

## Training Loop

In [ ]:
signiture = 'MNIST'
output_dim = 10

max_rank = 2
max_rank_list = [2,4,8,10,20]
max_rank_list = [4]
run = 1
# runs = 2

In [ ]:
for max_rank in max_rank_list:
#  for run in range(runs):

  model_path = os.path.join(model_path_dir, "model_{}_{}_{}.pt".format(signiture,max_rank,run))
  print("Temp location for models: {}".format(model_path))
  os.makedirs(os.path.dirname(model_path), exist_ok=True)
  best_valid_acc = 0
  best_valid_loss = float('inf')
  model = OrthoTONN(tensor_type='TensorTrainMatrix',max_rank=max_rank,dropouts=0.5,prior_type='log_uniform',eta=1.0, device=device,dtype=dtype)
  if cuda == 1:
      model = model.cuda()
      DTYPE = torch.cuda.FloatTensor
      LONG = torch.cuda.LongTensor

  # criterion = nn.CrossEntropyLoss(reduction='sum') 
  # factors = list(model.parameters())[:3]
  # other = list(model.parameters())[3:]
  # optimizer = optim.Adam([{"params": factors, "lr": factor_lr}, {"params": other, "lr": lr}], weight_decay=decay)

  criterion = nn.CrossEntropyLoss(reduction='sum') 
  optimizer = optim.Adam(list(model.parameters()), lr, weight_decay=decay)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)    # 3

  comment = f'rank = {max_rank} run = {run}'
  writer = SummaryWriter(comment=comment, log_dir='./tt_runs')

  for e in range(epochs):
    # TRAIN
    size = len(train_dataloader.dataset)
    model.train()
    model.zero_grad()
    avg_train_loss = 0.0

    for batch, (X, y) in enumerate(train_dataloader):
      X, y = X.to(device), y.to(device)

      # Compute prediction error
      pred = model(X)
      loss = criterion(pred, y)

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      avg_train_loss += loss.data.item()
      optimizer.step()
    
    # len(training_data) = len(train_dataloader.dataset) -- how many samples
    # len(train_dataloader) -- how many batches
    avg_train_loss = avg_train_loss / len(training_data)  # average over whole training dataset
    print("Epoch {} complete! Average Training loss: {}".format(e, avg_train_loss))
    writer.add_scalar("Avg Train Loss", avg_train_loss, e)

    # Terminate the training process if run into NaN
    if np.isnan(avg_train_loss):
        print("Training got into NaN values...\n\n")
        complete = False
        break
    
    #Valid/Test
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, acc_score, f1 = 0, 0, 0
    avg_valid_loss = 0.0
    with torch.no_grad():
      for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        avg_valid_loss += criterion(pred, y).item()
        
        # y itself is true label
        all_true_label = y.cpu().data
        all_predicted_label = pred.argmax(1).cpu().data
        acc_score += accuracy_score(all_true_label, all_predicted_label)
        f1 += f1_score(all_true_label, all_predicted_label, average='weighted')

        # argmax will eliminate this dimension!
        # correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    # in every iteration, the loss of the batch is summed up. actual loss should divide dataset size
    avg_valid_loss = avg_valid_loss / len(test_data)  # average over whole validation dataset
    print("Validation loss is: {}".format(avg_valid_loss))
    writer.add_scalar("Avg Valid Loss", avg_valid_loss, e)

    if np.isnan(avg_valid_loss):
      print("Training got into NaN values...\n\n")
      complete = False
      break

    # score is calculated every batch
    acc_score /= num_batches
    f1 /= num_batches

    print("Accuracy score is {}".format(acc_score))
    print("F1-score is {}".format(f1))
    
    if (avg_valid_loss < best_valid_loss):
        curr_patience = patience
        best_valid_loss = avg_valid_loss
        torch.save(model, model_path)
        print("Found new best model, saving to disk...")
    else:
        curr_patience -= 1
    
    if curr_patience <= 0:
        break
        

In [ ]:
X = torch.reshape(X,[64,1,28,28])
model.eval()
X, y = X.to(device), y.to(device)
pred = model(X)
pred.size(), y.size()

In [ ]:
all_true_label = y.cpu().data
all_predicted_label = pred.argmax(1).cpu().data
print("all_true_label is: {}".format(all_true_label))
print("all_predicted_label is: {}".format(all_predicted_label))

In [ ]:
pred.argmax(1).cpu().data.size()

## Eval Best Model

In [ ]:
max_rank = 2
run = 1
model_path = os.path.join(model_path_dir, "model_{}_{}_{}.pt".format(signiture,max_rank,run))
best_model = torch.load(model_path)
best_model.eval()
best_model.to('cpu')

In [ ]:
best_model.fc1.tensor.trainable_variables[2].data.size()

torch.Size([2, 7, 8, 2])

In [ ]:
order = best_model.fc1.tensor.order
ortho_cores = []

for i_order in range(order-1,-1,-1):

  tt_core = best_model.fc1.tensor.trainable_variables[i_order].data
  size = tt_core.size()
  tt_matrix = torch.reshape(tt_core, (size[0]*size[1], size[2]*size[3]))
  if i_order != order-1:
    tt_core = torch.matmul(tt_matrix, L)
  
  tt_matrix = torch.transpose(tt_matrix,0,1)
  QT, R = torch.linalg.qr(tt_matrix, mode="complete")
  L = torch.transpose(R,0,1)
  Q = torch.transpose(Q,0,1)
  ortho_cores.insert(0,Q)

U, S, Vh = torch.linalg.svd(L)
ortho_cores.insert(0,Vh)
ortho_cores.insert(0,U)

RuntimeError: ignored

In [ ]:
print(tt_matrix.size())
print(Q)
print(R)

torch.Size([8, 4])
tensor([[-0.4715,  0.1050,  0.1914, -0.3383,  0.0488,  0.5277, -0.5783,  0.0174],
        [-0.2078, -0.1173,  0.4082,  0.5203,  0.2509, -0.4624, -0.4283, -0.2135],
        [-0.0342,  0.0886, -0.3271, -0.5492,  0.2030, -0.6334, -0.2971,  0.2273],
        [-0.1245,  0.2322,  0.6431, -0.4141, -0.3474, -0.2737,  0.3126, -0.2286]])
tensor([[-0.6149, -0.4502, -0.5039, -0.5871],
        [ 0.0000,  0.1646,  0.1201,  0.2936],
        [ 0.0000,  0.0000, -0.0480, -0.0273],
        [ 0.0000,  0.0000,  0.0000, -0.1658]])


In [ ]:
from .interferometer import interferometer as itf 
Q = ortho_cores[-1]
I = itf.square_decomposition(Q)
I.draw()

3
2
1
0


In [ ]:
def eval_at_model_path(model_path, test_iterator):
  best_model = torch.load(model_path)
  best_model.eval()
  best_model.to(device)

  #Valid/Test
  size = len(test_iterator.dataset)
  num_batches = len(test_iterator)
  test_loss, acc_score, f1 = 0, 0, 0
  avg_valid_loss = 0.0
  with torch.no_grad():
    for X, y in test_iterator:
      X, y = X.to(device), y.to(device)
      pred = best_model(X)
      
      # y itself is true label
      all_true_label = y.cpu().data
      all_predicted_label = pred.argmax(1).cpu().data
      acc_score += accuracy_score(all_true_label, all_predicted_label)
      f1 += f1_score(all_true_label, all_predicted_label, average='weighted')

  # score is calculated every batch
  acc_score /= num_batches
  f1 /= num_batches
  print("Accuracy score is {}".format(acc_score))
  print("F1-score is {}".format(f1))

  return acc_score

In [ ]:
best_model_path = 0
best_acc = 0
for max_rank in max_rank_list:
#  for run in range(runs):
  model_path = os.path.join(model_path_dir, "model_{}_{}_{}.pt".format(signiture,max_rank,run))
  print(model_path)
  acc = eval_at_model_path(model_path, test_dataloader)
  print("-------------")
  if acc > best_acc:
      best_acc = acc
      best_model_path = model_path

./tt_models/model_MNIST_2_1.pt
Accuracy score is 0.9357085987261147
F1-score is 0.9351699481399507
-------------
./tt_models/model_MNIST_4_1.pt
Accuracy score is 0.962281050955414
F1-score is 0.9619681106624476
-------------
./tt_models/model_MNIST_8_1.pt
Accuracy score is 0.9735270700636943
F1-score is 0.9733664613900941
-------------
